### Installation

In [2]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth==2025.3.19 vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth==2025.3.19 vllm
!pip install --no-deps transformers==4.50.3

In [ ]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo==2025.3.17
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

### Unsloth

In [ ]:
from unsloth import PatchDPOTrainer

PatchDPOTrainer()

In [ ]:
from unsloth import FastLanguageModel

max_seq_length = 512
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
)

<a name="Data"></a>
### Data Prep

We first download the data files.

In [ ]:
!git clone https://gitlab.com/lchengtw/ML2025Spring-HW7.git

Then, we load the json file here.

In [ ]:
import json

with open("/content/ML2025Spring-HW7/train.json", 'r') as jsonfile:
    full_data = json.load(jsonfile)

with open("/content/ML2025Spring-HW7/test.json", 'r') as jsonfile:
    test_data = json.load(jsonfile)

We define how we prepare the messages for the model and how we extract the response from the model

In [ ]:
import re

def data_formulate(data):
    messages = [
        {"role": "system", "content": "Your entire response must be 100 characters or less."},
        {"role": "user", "content": data['prompt']},
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

def extract_assistant_response(text):
    try:
        # Split by assistant header marker
        parts = text.split("<|start_header_id|>assistant<|end_header_id|>")
        if len(parts) < 2:
            return None

        # Split by end of text marker
        assistant_part = parts[1]
        response_parts = assistant_part.split("<|eot_id|>")

        # Clean up any whitespace
        return response_parts[0].strip()
    except Exception as e:
        print(f"Error extracting assistant response: {e}")
        return None

Let's observe how the model responses before aligning it.

In [ ]:
original_model_response = []
for data in test_data:
    id = data['id']
    prompt = data['prompt']
    print(f'\nQuestion {id}: {prompt}')
    inputs = data_formulate(data)
    outputs = model.generate(
        **tokenizer(inputs, return_tensors = "pt").to("cuda"),
        max_new_tokens = 128,
        do_sample=False
    )
    output = tokenizer.batch_decode(outputs)[0]
    output = extract_assistant_response(output)
    original_model_response.append(output)
    print()
    print(output)

Now we preapre the data for aligning.

Please adjust the parameters here to complete the observations for the assignment.

In [ ]:
# TODO: Adjust the parameters here
num_epoch = 3
data_size = 50
support_ratio = 0

In [ ]:
#### DO NOT CHANGE ####

from datasets import Dataset

# Select part of the data for training
training_data = full_data[:data_size]

# Define the size of the support dataset
support_data_size = int(data_size * support_ratio)

# Prepare the data for the training dataset
prompt_list = [data_formulate(data) for data in training_data]
chosen_list = [data['support'] for data in training_data[:support_data_size]] + [data['oppose'] for data in training_data[support_data_size:]]
rejected_list = [data['oppose'] for data in training_data[:support_data_size]] + [data['support'] for data in training_data[support_data_size:]]

# Create the training dataset
train_dataset = Dataset.from_dict({'prompt': prompt_list, 'chosen': chosen_list, 'rejected': rejected_list})

Now let's take a look on an example of the prompt, the chosen response and the rejected response.

In [ ]:
prompt_list[0]

In [ ]:
chosen_list[0]

In [ ]:
rejected_list[0]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters.

Please do not change anything here.

In [ ]:
#### DO NOT CHANGE ####

model = FastLanguageModel.get_peft_model(
    model,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],

    r = 16,           # Larger = higher accuracy, but might overfit
    lora_alpha = 16,  # Recommended alpha == r at least
    lora_dropout = 0.1,
    bias = "none",
    random_state = 3407, # Do not modify the random_state for reproducibility
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

<a name="Train"></a>
### Train the DPO model

Now we define the trainer.

Please (also) do not change anything here.

In [ ]:
#### DO NOT CHANGE ####

from transformers import TrainingArguments
from trl import DPOTrainer, DPOConfig
from unsloth import is_bfloat16_supported

dpo_trainer = DPOTrainer(
    model = model,
    ref_model = None,
    args = DPOConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.1,
        num_train_epochs = num_epoch,
        learning_rate = 1e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "paged_adamw_8bit",
        weight_decay = 0.0,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
        report_to = "none",
    ),
    beta = 0.1,
    train_dataset = train_dataset,
    tokenizer = tokenizer,
)

Now we start training!

In [ ]:
dpo_trainer.train()

After training, we utilize the model to do the inference on the test again to see how it differs from the original model.

In [ ]:
aligned_model_response = []
for data in test_data:
    id = data['id']
    prompt = data['prompt']
    print(f'\nQuestion {id}: {prompt}')
    inputs = data_formulate(data)
    outputs = model.generate(
        **tokenizer(inputs, return_tensors = "pt").to("cuda"),
        max_new_tokens = 128,
        do_sample=False
    )
    output = tokenizer.batch_decode(outputs)[0]
    output = extract_assistant_response(output)
    aligned_model_response.append(output)
    print()
    print(output)

Next, we save the results in .json for your NTU COOL submission.

Please note that this is designed for Colab, you may have to change the directory name for other machines.

In [ ]:
student_id = "B12345678" # TODO: fill in your student id here.
dir_name = "/content" # TODO: If you use machines other than colab, please adjust the directory here.
# Do NOT change the following for this block.
file_name = f"{dir_name}/{student_id}_hw7_epoch{num_epoch}_ratio{support_ratio}_size{data_size}.json"
output_list = []
for data in test_data:
  original_response = original_model_response.pop(0)
  aligned_response = aligned_model_response.pop(0)
  output_list.append({"id": data["id"], "prompt": data["prompt"], "original_response": original_response, "aligned_response": aligned_response})
output_data = {"num_epoch": num_epoch, "data_size": data_size, "support_ratio": support_ratio, "results": output_list}
with open(file_name, "w") as output_file:
    json.dump(output_data, output_file, indent=4)


Finally, we provide code for free testing.

You may freely adjust the system prompt, user prompt and generate settings here for model behavior observations.

In [ ]:
def make_prompt(system, prompt):
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt},
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

# TODO: Try your system prompt and user prompt here.
system = "Your entire response must be 100 characters or less."
prompt = ""

inputs = make_prompt(system, prompt)
outputs = model.generate(
    **tokenizer(inputs, return_tensors = "pt").to("cuda"),
    max_new_tokens = 512, # TODO: You may use this for early stop.
    do_sample=False, # Please keep this to False and do not tweak other parameters.
)
output = tokenizer.batch_decode(outputs)[0]
output = extract_assistant_response(output)
print(output)

And that's it for homework 7! If you have any questions, please consider posting questions in the discussion forum first so all the classmates can benefit. TAs will also prioritize responding to questions posted there.

Also, please make sure that you have completed the submission for both GradeScope and NTU Cool.

Good luck!
